In [ ]:
from pathlib import Path

from lightglue import LightGlue, SuperPoint, DISK
from lightglue.utils import load_image, rbd
from lightglue import viz2d
import torch
import os

torch.set_grad_enabled(False)
images = Path("assets")

## Load extractor and matcher module
In this example we use SuperPoint features combined with LightGlue.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # 'mps', 'cpu'

extractor = SuperPoint(max_num_keypoints=2048).eval().to(device)  # load the extractor
matcher = LightGlue(features="superpoint").eval().to(device)

## Easy example
The top image shows the matches, while the bottom image shows the point pruning across layers. In this case, LightGlue prunes a few points with occlusions, but is able to stop the context aggregation after 4/9 layers.

In [ ]:
image0 = load_image("/home/oop/dev/data/1c5fd88d-d6c0-4ca4-a731-4373f554e819_3of10.png")
image1 = load_image("/home/oop/dev/data/1c5fd88d-d6c0-4ca4-a731-4373f554e819_5of10.png")
image0 = load_image("/home/oop/dev/data/1c5fd88d-d6c0-4ca4-a731-4373f554e819_7of10.png")

feats0 = extractor.extract(image0.to(device))
feats1 = extractor.extract(image1.to(device))
matches01 = matcher({"image0": feats0, "image1": feats1})
feats0, feats1, matches01 = [
    rbd(x) for x in [feats0, feats1, matches01]
]  # remove batch dimension

kpts0, kpts1, matches = feats0["keypoints"], feats1["keypoints"], matches01["matches"]
m_kpts0, m_kpts1 = kpts0[matches[..., 0]], kpts1[matches[..., 1]]

axes = viz2d.plot_images([image0, image1])
viz2d.plot_matches(m_kpts0, m_kpts1, color="lime", lw=0.2)
viz2d.add_text(0, f'Stop after {matches01["stop"]} layers', fs=20)

kpc0, kpc1 = viz2d.cm_prune(matches01["prune0"]), viz2d.cm_prune(matches01["prune1"])
viz2d.plot_images([image0, image1])
viz2d.plot_keypoints([kpts0, kpts1], colors=[kpc0, kpc1], ps=10)

## Difficult example
For pairs with significant viewpoint- and illumination changes, LightGlue can exclude a lot of points early in the matching process (red points), which significantly reduces the inference time.

In [ ]:
image0 = load_image("/home/oop/dev/data/1c5fd88d-d6c0-4ca4-a731-4373f554e819_3of10.png")
# image1 = load_image("/home/oop/dev/data/1c5fd88d-d6c0-4ca4-a731-4373f554e819_5of10.png")
image0 = load_image("/home/oop/dev/data/1c5fd88d-d6c0-4ca4-a731-4373f554e819_7of10.png")

feats0 = extractor.extract(image0.to(device))
feats1 = extractor.extract(image1.to(device))
matches01 = matcher({"image0": feats0, "image1": feats1})
feats0, feats1, matches01 = [
    rbd(x) for x in [feats0, feats1, matches01]
]  # remove batch dimension

kpts0, kpts1, matches = feats0["keypoints"], feats1["keypoints"], matches01["matches"]
m_kpts0, m_kpts1 = kpts0[matches[..., 0]], kpts1[matches[..., 1]]

axes = viz2d.plot_images([image0, image1])
viz2d.plot_matches(m_kpts0, m_kpts1, color="lime", lw=0.2)
viz2d.add_text(0, f'Stop after {matches01["stop"]} layers')

kpc0, kpc1 = viz2d.cm_prune(matches01["prune0"]), viz2d.cm_prune(matches01["prune1"])
viz2d.plot_images([image0, image1])
viz2d.plot_keypoints([kpts0, kpts1], colors=[kpc0, kpc1], ps=6)

In [ ]:
print(kpts0)
print(kpts1)
print("kpts0", kpts0.shape)
print("kpts1", kpts1.shape)

print("matches", matches)
print("matches", matches.shape)

print("matches01", matches01)
print("matches01['matching_scores0']", matches01["matching_scores0"].shape)
print("matches01['matching_scores1']", matches01["matching_scores1"].shape)

In [ ]:
# Get the indices of the matches from the first and second images
match_indices0 = matches[:, 0]
match_indices1 = matches[:, 1]

# Use these indices to filter the matching_scores tensors
matching_scores0 = matches01['matching_scores0'][match_indices0]
matching_scores1 = matches01['matching_scores1'][match_indices1]

# Print the filtered matching scores
print("Filtered matching_scores0", matching_scores0)
print("Filtered matching_scores1", matching_scores1)

In [ ]:
import matplotlib.pyplot as plt

# Create a scatter plot
plt.figure(figsize=(8, 6))
plt.scatter(matching_scores0.cpu().numpy(), matching_scores1.cpu().numpy(), alpha=0.5)

# Set the axes limits
plt.xlim([0, 1])
plt.ylim([0, 1])

# Set the labels
plt.xlabel('Matching Scores 0')
plt.ylabel('Matching Scores 1')

# Show the plot
plt.show()

In [ ]:
import itertools
from pathlib import Path
import matplotlib.pyplot as plt

# Specify the UUID
uuid = "1c5fd88d-d6c0-4ca4-a731-4373f554e819"

# Get all image paths in the data directory that match the pattern
image_paths = sorted(list(Path("/home/oop/dev/data").glob(f"{uuid}*")))

# Iterate over all pairs of images
for image_path1, image_path2 in itertools.combinations(image_paths, 2):
    # Load the images
    image1 = load_image(str(image_path1))
    image2 = load_image(str(image_path2))

    # Extract features and match
    feats1 = extractor.extract(image1.to(device))
    feats2 = extractor.extract(image2.to(device))
    matches12 = matcher({"image0": feats1, "image1": feats2})
    feats1, feats2, matches12 = [rbd(x) for x in [feats1, feats2, matches12]]

    # Get keypoints and matches
    kpts1, kpts2, matches = feats1["keypoints"], feats2["keypoints"], matches12["matches"]
    m_kpts1, m_kpts2 = kpts1[matches[..., 0]], kpts2[matches[..., 1]]

    # Plot images and matches
    axes = viz2d.plot_images([image1, image2])
    viz2d.plot_matches(m_kpts1, m_kpts2, color="lime", lw=0.2)
    viz2d.add_text(0, f'Stop after {matches12["stop"]} layers')

    # Get the indices of the matches from the first and second images
    match_indices0 = matches[:, 0]
    match_indices1 = matches[:, 1]

    # Use these indices to filter the matching_scores tensors
    matching_scores0 = matches12['matching_scores0'][match_indices0]
    matching_scores1 = matches12['matching_scores1'][match_indices1]


    # Create a scatter plot
    plt.figure(figsize=(8, 6))
    plt.scatter(matching_scores0.cpu().numpy(), matching_scores1.cpu().numpy(), alpha=0.5)

    # print out average matching score
    print("Average matching score", (matching_scores0.cpu().numpy() + matching_scores1.cpu().numpy()) / 2)


    # Add the average matching score as a red dot
    plt.scatter(
        (matching_scores0.cpu().numpy() + matching_scores1.cpu().numpy()) / 2,
        (matching_scores0.cpu().numpy() + matching_scores1.cpu().numpy()) / 2,
        color="red",
    )

    # Set the axes limits
    plt.xlim([0, 1])
    plt.ylim([0, 1])

    # Set the labels
    plt.xlabel('Matching Scores 0')
    plt.ylabel('Matching Scores 1')

    # Show the plot
    plt.show()